The $k$-Means Algorithm
====================




The k-means algorithm is used to partition a given set of observations into a predefined amount of $k$ clusters. The algorithm as described by MacQueen in 1967, starts with a random set of $k$ center-points ($\mu$). During each update step, all observations $x$ are assigned to their nearest center-point. In the standard algorithm, only one assignment to one center is possible. If multiple centers have the same distance to the observation, a random one would be chosen.

$S_i^{(t)} = \big \{ x_p : \big \| x_p - \mu^{(t)}_i \big \|^2 \le \big \| x_p - \mu^{(t)}_j \big \|^2 \ \forall j, 1 \le j \le k \big\}$


Afterwards, the center-points are repositioned by calculating the mean of the assigned observations to the respective center-points.


$\mu^{(t+1)}_i = \frac{1}{|S^{(t)}_i|} \sum_{x_j \in S^{(t)}_i} x_j$


In [ ]:
from matplotlib import pyplot as plt
import numpy as np

We start by generating some artificial data:

In [ ]:
plt.jet() # set the color map. When your colors are lost, re-run this.
import sklearn.datasets as datasets
X, Y = datasets.make_blobs(centers=4, cluster_std=0.5, random_state=0)

As always, we first *plot* the data to get a feeling of what we're dealing with:

In [ ]:
plt.scatter(X[:,0], X[:,1])
plt.show()

The data looks like it may contain four different "types" of data point. 

In fact, this is how it was created above.

We can plot this information as well, using color:

In [ ]:
plt.scatter(X[:,0], X[:,1], c=Y)
plt.show()

Normally, you do not know the information in `Y`, however.

You could try to recover it from the data alone.

This is what the kMeans algorithm does. 

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(4, random_state=8)
Y_hat = kmeans.fit(X).labels_

Now the label assignments should be quite similar to `Y`, up to a different ordering of the colors:

In [ ]:
plt.scatter(X[:,0], X[:,1], c=Y_hat)
plt.show()

Often, you're not so much interested in the assignments to the means. 

You'll want to have a closer look at the means $\mu$.

The means in $\mu$ can be seen as *representatives* of their respective cluster.

In [ ]:
plt.scatter(X[:,0], X[:,1], c=Y_hat, alpha=0.4)
mu = kmeans.cluster_centers_
plt.scatter(mu[:,0], mu[:,1], s=100, c=np.unique(Y_hat))
print(mu)
plt.show()

## $k$-Means on Images

In this final example, we use the $k$-Means algorithm on the classical MNIST dataset.

The MNIST dataset contains images of hand-written digits. 

Let's first fetch the dataset from the internet (which may take a while, note the asterisk [*]):

In [ ]:
from sklearn.datasets import load_digits

digits = load_digits()

In [ ]:
digits.images[0].shape

In [ ]:
from sklearn.datasets import fetch_mldata
from sklearn.cluster import KMeans
from sklearn.utils import shuffle
X_digits, Y_digits = (digits.data, digits.target)

Let's have a look at some of the instances in the dataset we just loaded:

In [ ]:
plt.rc("image", cmap="binary") # use black/white palette for plotting
for i in range(10):
    plt.subplot(2,5,i+1)
    plt.imshow(X_digits[i].reshape(8,8))
    plt.xticks(())
    plt.yticks(())
plt.tight_layout()
plt.show()

**Warning**: This takes quite a few seconds, so be patient until the asterisk [*] disappears!

In [ ]:
kmeans = KMeans(10)
mu_digits = kmeans.fit(X_digits).cluster_centers_

Let's have a closer look at the means. Even though there are 10 digits, some of them are over/under-represented. Do you understand why?

In [ ]:
plt.figure(figsize=(6,6))
for i in range(2*(mu_digits.shape[0]//2)): # loop over all means
    plt.subplot(2,mu_digits.shape[0]//2,i+1)
    plt.imshow(mu_digits[i].reshape(8,8))
    plt.xticks(())
    plt.yticks(())
plt.tight_layout()
plt.show()

## Playing with $k$-Means - Advanced visualization

We're now going to apply PCA to the dataset and try to get a feel for the distribution of each of the handwritten digits.

In [ ]:
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale

### Now fit another KMeans classifier on 2D PCA data

In [ ]:
reduced_data = PCA(n_components=2).fit_transform(X_digits)
kmeans = KMeans(init='k-means++', n_clusters=10, n_init=10)
kmeans.fit(reduced_data)

In [ ]:
# Plot the decision boundary. For that, we will assign a color to each
h = .02     # point in the mesh [x_min, x_max]x[y_min, y_max].

x_min, x_max = reduced_data[:, 0].min() - 1, reduced_data[:, 0].max() + 1
y_min, y_max = reduced_data[:, 1].min() - 1, reduced_data[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

# Obtain labels for each point in mesh. Use last trained model.
Z = kmeans.predict(np.c_[xx.ravel(), yy.ravel()])

In [ ]:
# Put the result into a color plot
Z = Z.reshape(xx.shape)
plt.figure(1, figsize=(12,12))
plt.clf()
plt.imshow(Z, interpolation='nearest',
           extent=(xx.min(), xx.max(), yy.min(), yy.max()),
           cmap=plt.cm.Paired,
           aspect='auto', origin='lower')

plt.plot(reduced_data[:, 0], reduced_data[:, 1], 'k.', markersize=2)
# Plot the centroids as a white X
centroids = kmeans.cluster_centers_
plt.scatter(centroids[:, 0], centroids[:, 1],
            marker='x', s=169, linewidths=3,
            color='w', zorder=10)
plt.title('K-means clustering on the digits dataset (PCA-reduced data)\n'
          'Centroids are marked with white cross')
plt.xlim(x_min, x_max)
plt.ylim(y_min, y_max)
plt.xticks(())
plt.yticks(())
plt.show()

Playing with this Notebook
==========================

Try to see what happens when you

- Increase the standard deviation of the clusters in this notebook
- Choose a "wrong" number of clusters by:
  1. changing the number of clusters generated
  2. changing the number of clusters used by KMeans

- What happens to result of the $k$-Means algorithm when you have multiplied one axis of the matrix $X$ with a large value?

  For example, the 0-th axis with 100:

  `X[:,0] *= 100`

  Why does the result change?
